# Cubes with inelastic volume strains
[![Google Collab Book](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/meyer-nils/torch-fem/blob/main/examples/basic/solid/inelasticity.ipynb)

All unit cubes are subjected to an inelastic (e.g. thermal) strain, while being fixed in one direction and allowing shrinkage in the two transverse directions.

In [1]:
import meshzoo
import torch

from torchfem import Solid
from torchfem.materials import IsotropicElasticity3D

torch.set_default_dtype(torch.float64)

# Elastic material model
material = IsotropicElasticity3D(E=1000.0, nu=0.3)

## 3D Cube with linear hexahedrons  and inelastic (e.g. thermal) strain

In [2]:
# Generate cube
points, cells = meshzoo.cube_hexa(
    torch.linspace(0.0, 1.0, 5),
    torch.linspace(0.0, 1.0, 5),
    torch.linspace(0.0, 1.0, 5),
)
nodes = torch.tensor(points)
elements = torch.tensor(cells.tolist())

# Create model
box = Solid(nodes, elements, material)

# Set constraints
box.constraints[:, :] = False
box.constraints[nodes[:, 0] == 0.0, 0] = True
box.constraints[nodes[:, 0] == 1.0, 0] = True
box.constraints[nodes[:, 1] == 0.5, 1] = True
box.constraints[nodes[:, 2] == 0.5, 2] = True

# Set inelastic strain (isotropic shrinkage)
s = -0.1
box.ext_strain = torch.stack(len(elements) * [torch.tensor([s, s, s, 0.0, 0.0, 0.0])])

u, f, σ, ε, α = box.solve()

box.plot(u=u, node_property={"Disp": u}, show_undeformed=True)

Widget(value='<iframe src="http://localhost:61747/index.html?ui=P_0x13deee4a0_0&reconnect=auto" class="pyvista…

In [3]:
print(f"Mean stress in x is {σ[:, 0].mean()}. It should be {-material.E * s}.")

Mean stress in x is 100.0. It should be 100.0.


## 3D Cube with linear tetrahedrons and inelastic (e.g. thermal) strain

In [4]:
# Generate cube
points, cells = meshzoo.cube_tetra(
    torch.linspace(0.0, 1.0, 3),
    torch.linspace(0.0, 1.0, 3),
    torch.linspace(0.0, 1.0, 3),
)
nodes = torch.tensor(points)
elements = torch.tensor(cells.tolist())

box = Solid(nodes, elements, material)

# Boundary conditions
box.constraints[nodes[:, 0] == 0.0, 0] = True
box.constraints[nodes[:, 0] == 1.0, 0] = True
box.constraints[nodes[:, 1] == 0.5, 1] = True
box.constraints[nodes[:, 2] == 0.5, 2] = True

# Set inelastic strain (isotropic shrinkage)
s = -0.1
box.ext_strain = torch.stack(len(elements) * [torch.tensor([s, s, s, 0.0, 0.0, 0.0])])

u, f, σ, ε, α = box.solve()

box.plot(u=u, node_property={"Disp": u}, show_undeformed=True)

Widget(value='<iframe src="http://localhost:61747/index.html?ui=P_0x128b773d0_1&reconnect=auto" class="pyvista…

In [5]:
print(f"Mean stress in x is {σ[:, 0].mean()}. It should be {-material.E * s}.")

Mean stress in x is 100.0. It should be 100.0.
